In [1]:
var verificatum = require('./js/min-vjsc-1.1.1.js').verificatum;
var fs = require('fs');

In [2]:
var WIDTH = 1;  // Depends on vmni configuration

In [3]:
// 1. Read 'PublicKey' file which is saved as a "raw" ByteTree
var data = fs.readFileSync("./publicKey");
var bt = verificatum.eio.ByteTree.readByteTreeFromByteArray([...data]);

In [4]:
assert(verificatum.util.byteArrayToAscii(bt.value[0].value[0].value).endsWith('ECqPGroup'));
var keyPGroup = verificatum.arithm.ECqPGroup.fromByteTree(bt.value[0].value[1])
keyPGroup

ECqPGroup {
  pRing: 
   PField {
     order: LargeInteger { sign: 1, value: [Array], length: 10 },
     bitLength: 256,
     byteLength: 33 },
  curve: 
   EC {
     modulus: LargeInteger { sign: 1, value: [Array], length: 10 },
     length: 24,
     a: LargeInteger { sign: 1, value: [Array], length: 10 },
     b: LargeInteger { sign: 1, value: [Array], length: 10 },
     jdbl_raw: [Function] },
  generator: 
   ECqPGroupElement {
     pGroup: [Circular],
     fixExp: null,
     expCounter: 0,
     value: ECP { x: [Object], y: [Object], z: [Object] } },
  ONE: 
   ECqPGroupElement {
     pGroup: [Circular],
     fixExp: null,
     expCounter: 0,
     value: ECP { x: [Object], y: [Object], z: [Object] } },
  modulusByteLength: 33,
  encodeLength: 28 }

In [5]:
var fullPublicKey = new verificatum.arithm.PPGroup(keyPGroup, 2).toElement(bt.value[1]);
console.log(fullPublicKey.toString());
fullPublicKey

((6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296,4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5),(6be64361c3e559ab3bfccfe370806570b8af68ee9a19feb5ea444f0728cb49b5,57a501113acac3dcf227475920bd966e9e0c575bff9b8f80d8bef6845db7eb69))


PPGroupElement {
  pGroup: 
   PPGroup {
     pRing: PPRing { pRings: [Array], ZERO: [Object], ONE: [Object], byteLength: 81 },
     pGroups: [ [Object], [Object] ],
     encodeLength: 56,
     generator: 
      PPGroupElement {
        pGroup: [Circular],
        fixExp: null,
        expCounter: 0,
        values: [Array] },
     ONE: 
      PPGroupElement {
        pGroup: [Circular],
        fixExp: null,
        expCounter: 0,
        values: [Array] },
     byteLength: 167 },
  fixExp: null,
  expCounter: 0,
  values: 
   [ ECqPGroupElement { pGroup: [Object], fixExp: null, expCounter: 0, value: [Object] },
     ECqPGroupElement { pGroup: [Object], fixExp: null, expCounter: 0, value: [Object] } ] }

In [6]:
// const basicPublicKey = fullPublicKey.project(0);
// const publicKey = fullPublicKey.project(1);
// const pRing = publicKey.pGroup.pRing;
// console.log(publicKey.toString());
// console.log(basicPublicKey.toString());
// console.log(pRing.toString());

In [7]:
// verificatum.arithm.PGroup.getPGroup("P-256")
var eg = new verificatum.crypto.ElGamal(true, keyPGroup, undefined, 0);

In [8]:
var wpk = eg.widePublicKey(fullPublicKey, WIDTH);
wpk

PPGroupElement {
  pGroup: 
   PPGroup {
     pRing: PPRing { pRings: [Array], ZERO: [Object], ONE: [Object], byteLength: 81 },
     pGroups: [ [Object], [Object] ],
     encodeLength: 56,
     generator: 
      PPGroupElement {
        pGroup: [Circular],
        fixExp: null,
        expCounter: 0,
        values: [Array] },
     ONE: 
      PPGroupElement {
        pGroup: [Circular],
        fixExp: null,
        expCounter: 0,
        values: [Array] },
     byteLength: 167 },
  fixExp: null,
  expCounter: 0,
  values: 
   [ ECqPGroupElement { pGroup: [Object], fixExp: null, expCounter: 0, value: [Object] },
     ECqPGroupElement { pGroup: [Object], fixExp: null, expCounter: 0, value: [Object] } ] }

In [10]:
var choices = ["Napoleon", "George Bush", "Christina, Queen of Sweden"];
var str_messages = [];
var tally = new Array(choices.length).fill(0);
for (var i = 0; i < 100; i++) {
    var idx = Math.floor(Math.random() * choices.length);
    str_messages[i] = choices[idx]
    tally[idx]++;
}

tally

[ 40, 39, 21 ]

In [11]:
var ascii_bytes = str_messages.map(x => x.split('').map(x=>x.charCodeAt(0)));  // TODO: verificatum utils

In [12]:
var encoded = ascii_bytes.map(x => wpk.pGroup.project(1).encode(x));
encoded

[ ECqPGroupElement {
    pGroup: 
     ECqPGroup {
       pRing: [Object],
       curve: [Object],
       generator: [Object],
       ONE: [Object],
       modulusByteLength: 33,
       encodeLength: 28 },
    fixExp: null,
    expCounter: 0,
    value: ECP { x: [Object], y: [Object], z: [Object] } },
  ECqPGroupElement {
    pGroup: 
     ECqPGroup {
       pRing: [Object],
       curve: [Object],
       generator: [Object],
       ONE: [Object],
       modulusByteLength: 33,
       encodeLength: 28 },
    fixExp: null,
    expCounter: 0,
    value: ECP { x: [Object], y: [Object], z: [Object] } },
  ECqPGroupElement {
    pGroup: 
     ECqPGroup {
       pRing: [Object],
       curve: [Object],
       generator: [Object],
       ONE: [Object],
       modulusByteLength: 33,
       encodeLength: 28 },
    fixExp: null,
    expCounter: 0,
    value: ECP { x: [Object], y: [Object], z: [Object] } },
  ECqPGroupElement {
    pGroup: 
     ECqPGroup {
       pRing: [Object],
       curve: [O

In [13]:
var encrypted = encoded.map(x => eg.encrypt(wpk, x));

In [14]:
// XXX: why?
var encrypted0 = encrypted.map(x => x.values[0].toByteTree());
var encrypted1 = encrypted.map(x => x.values[1].toByteTree());
var encrypted_bytetree = new verificatum.eio.ByteTree([new verificatum.eio.ByteTree(encrypted0), new verificatum.eio.ByteTree(encrypted1)])

In [15]:
fs.writeFileSync('ciphertexts', new Buffer(encrypted_bytetree.toByteArray()))

In [16]:
// var res_data = fs.readFileSync("./plaintexts");
// var res_bt = verificatum.eio.ByteTree.readByteTreeFromByteArray([...res_data]);
// res_bt.value.map(x => verificatum.util.byteArrayToAscii(x.value[0].value.filter(c => c != 0)))